In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import rasterio
import xml.etree.ElementTree as ET

In [6]:
## parse the XML label files
tree = ET.parse('data2020/venus_2020_north_ocp.xml')
root = tree.getroot()

In [71]:
## doesn't work. fml bruce. thanks for using same tags for multiple things
label_dict = {}


for element in root.iter():
    label_dict[element.tag.split('}')[1]] = element.text
#         print(element.tag.split('}')[1] , element.attrib,element.text)
    if element.attrib != {}:
#             print(element.items())
        for item in element.items():
            label_dict[element.tag.split('}')[1]+"_"+item[0]] = item[1]
#                 print(element.tag.split('}')[1]+"_"+item[0])
#                 print(element.tag.split('}')[1]+"_"+item[1])
#                 print(item)
#             attrib_keys = element.keys()
#             attrib_values = element.values()
#             if len(attrib_keys) == 1:
#                 print(element.tag.split('}')[1]+"_"+attrib_keys[0])

print(len(label_dict))

102


In [20]:
lines = 12000
samples = 20000
filename = 'data2020/venus_2020_north_inc.img'
datatype = np.float32
f = open(filename, 'rb')
data_raw = np.fromfile(f, datatype)
data = data_raw.reshape(lines, samples)

In [25]:
print(np.max(data))

90.0


In [27]:
outfile = "data2020/north_inc.tif"
outds = rasterio.open(outfile, 'w', driver='GTiff', 
                  height = lines, 
                  width = samples, 
                  count=1, 
                  crs = None, 
                  dtype = np.float32,
                  transform = None,
                  compress='lzw',
                  nodata = 9999)
outds.write(data, 1)

In [56]:
act = {}
act['a'] = "abc"
act['d'] = "def"
print(act)

{'a': 'abc', 'd': 'def'}
